# Spectral Plots

Notes
- some lines when too close are not labeled (minLineSep)
- TOOD: x1 and x2 are not the same, but shifted so might need to interpolate? Check shifting carefully
- consistent colors for kronos and krios across *all* plots

In [ ]:
import palettable

In [ ]:
palettable.colorbrewer.qualitative.Paired_12.show_discrete_image()

In [ ]:
from __future__ import division # to get float division with ints

# Import readsav so we can read IDL Save Files
import warnings
from scipy.io import readsav
warnings.filterwarnings('ignore', message="warning: empty strings")

import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import re

In [ ]:
plt.style.use('notebook.mplstyle')

In [ ]:
import specplot

In [ ]:
krios = specplot.SpecPlot('../data/240429_rj211.86/iter1_abund.out')
kronos = specplot.SpecPlot('../data/240430_rj211.85/iter1_abund.out')

In [ ]:
segIdx = krios.getGoodSegs('Na')
markLines = krios.getLinesToMark(10, ['Na 1', 'Fe 1', 'Si 1', 'Ca 1'], minDepth=0)

In [ ]:
seg, wrange, elems = 9, (6154.22-0.5, 6154.22+2), ['Na 1', 'Fe 1', 'Si 1', 'Ca 1']
seg, wrange, elems = 10, (6160.7487-0.5, 6161.7), ['Na 1', 'Fe 1', 'Si 1', 'Ca 1']

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df['species'] = [s.decode() for s in krios.spec.species[0]]
df['depth'] = krios.spec.depth[0]
df['wave'] = krios.spec.atomic[0][:,2]

In [ ]:
def get_segidx(wave):
    idx = np.where((krios.wran[:,0] <= wave) & (wave <= krios.wran[:,1]))[0]
    if idx.size == 0:
        return -1
    return idx[0]
segidx = list(map(get_segidx, df.wave))
df['segidx'] = segidx

In [ ]:
elements = ['C', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'Ca',
            'Ti', 'V', 'Cr', 'Mn', 'Ni', 'Y']

In [ ]:
df['elem'] = df.species.str.extract('([a-zA-z0-9]+) [1-9]')
print(df.elem.describe())

In [ ]:
in_catalog = np.array([elem in elements for elem in df.elem])

In [ ]:
df_in_catalog = df.loc[in_catalog]

In [ ]:
df_in_catalog.loc[df_in_catalog.depth>0.25].elem.value_counts()

In [ ]:
df.loc[(df.segidx == 0) & (df.depth>0.25) & in_catalog].sort_values('depth').species.values

In [ ]:
4, [6016.67-1, 6016.67+1], ['Mn 1', 'Fe 1']

In [ ]:
krios_model_color = '#1F78B4'
krios_data_color = '#A6CEE3'
kronos_model_color = '#E31A1C'
kronos_data_color = '#FB9A99'


def plotKKSpec(ax, seg, wrange, elems, minDepth=0.25):
    from matplotlib.ticker import MaxNLocator
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    if len(elems)>0:
        markLines=krios.getLinesToMark(seg, elems, minDepth=minDepth)
    else:
        markLines = []

    krios.plotSeg(seg, wrange, showLines=markLines, ax=ax, labelSpeciesOnly=True,
                  modColor=krios_model_color, obsColor=krios_data_color,
                  modWidth=1);
    kronos.plotSeg(seg, wrange, showLines=markLines, ax=ax, labelSpeciesOnly=True,
                   modColor=kronos_model_color, obsColor=kronos_data_color,
                   modWidth=1);

    divider = make_axes_locatable(ax)
    ax_diff = divider.new_vertical(size="100%", pad=0.1, pack_start=True)
    fig.add_axes(ax_diff)
    # TODO: x1 and x2 are not the same, but shifted so might need to interpolate?
    x1, y1 = krios.lineObs.get_data()
    x2, y2 = kronos.lineObs.get_data()
    xm1, ym1 = krios.lineModel.get_data()
    xm2, ym2 = kronos.lineModel.get_data()
    # ax_diff.plot(x1, y2-y1, 'k.-', xm1, ym2-ym1, 'b-')
    ax_diff.plot(x1, y2/y1, '-', color='#999999')
    ax_diff.plot(xm1, ym2/ym1, 'k-', lw=1)
    ax_diff.axhline(0, ls='-', c='k', zorder=-50)
    ax.set_xticklabels([])
    ax_diff.set_xlim(ax.get_xlim())
    ax_diff.get_xaxis().get_major_formatter().set_useOffset(False)
    ax_diff.set_ylim(0.8,1.1)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=5, prune='lower'))
    ax_diff.yaxis.set_major_locator(MaxNLocator(nbins=5, prune='lower'))
    return ax, ax_diff

In [ ]:
plt.legend?

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,10), dpi=150)
fig.subplots_adjust(bottom=0.07, left=0.15, top=0.92, right=0.95, hspace=0.12)

elems = df.loc[(df.segidx == 9) & (df.depth>0.5) & (in_catalog)].sort_values('depth').species.values

cax0, cax_diff = plotKKSpec(ax[0], 0, krios.wran[0], [])
list(map(lambda x: x.set_xticks([5165, 5170, 5175, 5180, 5185]), [cax0, cax_diff]));
cax, cax_diff = plotKKSpec(ax[1], 1, krios.wran[1], [])
list(map(lambda x: x.set_xticks([5195, 5200, 5205]), [cax, cax_diff]));
cax, cax_diff = plotKKSpec(ax[2], 2, krios.wran[2], [])

cax_diff.set_xlabel('wavelength ($\AA$)')
fig.text(0.05, 0.55, "Residual intensity (ratio)",
         rotation=90, va='center', ha='center', size=25)

handles=[
    plt.Line2D([],[], linewidth=2, color=krios_data_color),
    plt.Line2D([],[], linewidth=1, color=krios_model_color),
    plt.Line2D([],[], linewidth=2, color=kronos_data_color),
    plt.Line2D([],[], linewidth=1, color=kronos_model_color)]
fig.legend(handles, ['Krios data', 'Krios model', 'Kronos data', 'Kronos model'],
           loc=(0.495, 0.932), ncol=2,
          )

fig.savefig('../paper/figures/spec1.pdf')

In [ ]:
in_catalog.sum()

In [ ]:
df.loc[(df.segidx == 4) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.values

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
cax.tick_params()

In [ ]:
plt.rcParams['axes.labelsize']

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), dpi=150)
elems = df.loc[(df.segidx == 4) & (df.depth>0.25) & (in_catalog)].species.unique()
elems = df.loc[(df.segidx == 4) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.unique()
cax, cax_diff = plotKKSpec(ax, 4, [6015.7,6024.5], ['Mn 1', 'Fe 1'], minDepth=0.1)
cax.xaxis.set_major_locator(MaxNLocator(2))

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
elems = df.loc[(df.segidx == 8) & (df.depth>0.25) & (in_catalog)].species.unique()
elems = df.loc[(df.segidx == 8) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.values
cax, cax_diff = plotKKSpec(ax, 8, [6121,6124], elems)
cax.set_ylim(cax.get_ylim()[0], 1.7)
cax_diff.xaxis.set_major_locator(MaxNLocator(2))

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

elems = df.loc[(df.segidx == 9) & (df.depth>0.5) & ((in_catalog)|(df.species=='Fe 1'))].species.values
cax, cax_diff = plotKKSpec(ax, 9, [6153, 6157], elems)
cax_diff.xaxis.set_major_locator(MaxNLocator(2))

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

elems = df.loc[(df.segidx == 14) & (df.depth>0.5) & ((in_catalog)|(df.species=='Fe 1'))].species.values
cax, cax_diff = plotKKSpec(ax, 9, [6153, 6157], elems)
cax_diff.xaxis.set_major_locator(MaxNLocator(2))

In [ ]:
plt.plot(krios.spec.wave[0])

In [ ]:
style = {
    'xtick.labelsize' : 15,
    'ytick.labelsize' : 15
}

with plt.style.context(style):
    fig = plt.figure(figsize=(10,6), dpi=100)
    fig.subplots_adjust(bottom=0.12, left=0.12, top=0.96, right=0.95, hspace=0.15)

    # First row
    ax = plt.subplot2grid((2, 12), (0, 0), colspan=3)
    elems = df.loc[(df.segidx == 8) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.values
    cax, cax_diff = plotKKSpec(ax, 8, [6121,6124], elems)
    cax.set_ylim(0.1, 1.8)
    list(map(lambda x: x.set_xticks([6122, 6123]), [cax, cax_diff]));
    
    ax = plt.subplot2grid((2, 12), (0, 3), colspan=3)
    elems = df.loc[(df.segidx == 9) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.values
    cax, cax_diff = plotKKSpec(ax, 9, [6153, 6157], elems)
    cax.set_ylim(0.1, 1.8)
    list(map(lambda x: x.set_xticks([6154, 6156]), [cax, cax_diff]));
    list(map(lambda x: x.set_yticklabels([]), [cax, cax_diff]));

    ax = plt.subplot2grid((2, 12), (0, 6), colspan=6)
    elems = df.loc[(df.segidx == 10) & (df.depth>0.25) & ((in_catalog))].species.unique()
    cax, cax_diff = plotKKSpec(ax, 10, [6160, 6170], elems)
    cax.set_ylim(0.1, 1.8)
    list(map(lambda x: x.set_xticks([6162, 6164, 6166, 6168]), [cax, cax_diff]));
    list(map(lambda x: x.set_yticklabels([]), [cax, cax_diff]));

    # Second row
    ax = plt.subplot2grid((2, 12), (1, 0), colspan=4)
    elems = df.loc[(df.segidx == 12) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.unique()
    cax, cax_diff = plotKKSpec(ax, 12, [6317.5, 6320], elems)
    cax.set_ylim(0.3, 1.6)
    list(map(lambda x: x.set_xticks([6318, 6319]), [cax, cax_diff]));
    
    ax = plt.subplot2grid((2, 12), (1, 4), colspan=4)
    seg, wrange = 14, [6695.5, 6700]
    elems = df.loc[(df.segidx == seg) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.unique()
    cax, cax_diff = plotKKSpec(ax, seg, wrange, elems)
    cax.set_ylim(0.3, 1.6)
    list(map(lambda x: x.set_xticks([6696, 6698]), [cax, cax_diff]));
    list(map(lambda x: x.set_yticklabels([]), [cax, cax_diff]));

    ax = plt.subplot2grid((2, 12), (1, 8), colspan=4)
    seg, wrange = 19, [7771, 7782]
    elems = df.loc[(df.segidx == seg) & (df.depth>0.25) & ((in_catalog)|(df.species=='Fe 1'))].species.unique()
    cax, cax_diff = plotKKSpec(ax, seg, wrange, elems)
    cax.set_ylim(0.3, 1.6)
    list(map(lambda x: x.set_xticks([7775, 7780]), [cax, cax_diff]));
    list(map(lambda x: x.set_yticklabels([]), [cax, cax_diff]));

    
    # cax, cax_diff = plotKKSpec(ax[1], 1, krios.wran[1], [])
    # cax_diff.set_xticks([5190, 5195, 5200, 5205])
    # cax, cax_diff = plotKKSpec(ax[2], 2, krios.wran[2], [])

#     cax_diff.set_xlabel('wavelength ($\AA$)')
    fig.text(0.55, 0.05, "wavelength ($\AA$)",
             va='center', ha='center', size=25)
    fig.text(0.04, 0.55, "Residual intensity (ratio)",
             rotation=90, va='center', ha='center', size=25)

    fig.savefig('../paper/figures/spec2.pdf')